In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split

from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('./team_26.csv')

data.head()

,Corner,vsup,temperature,l55lp_v0101.lib.scs,l55lp_v0101.lib.scs.1,l55lp_v0101.lib.scs.2,Pass/Fail,"VDC(""/net13"")"
0,C0_0,1.6,-55,ss_lp_io25,ff_lp_rvt12,ff_lp_bjt,NaN,0.6249
1,C0_1,1.6,-50,ss_lp_io25,ff_lp_rvt12,ff_lp_bjt,NaN,0.6136
2,C0_2,1.6,-40,ss_lp_io25,ff_lp_rvt12,ff_lp_bjt,NaN,0.5908
3,C0_3,1.6,-30,ss_lp_io25,ff_lp_rvt12,ff_lp_bjt,NaN,0.5680
4,C0_4,1.6,-20,ss_lp_io25,ff_lp_rvt12,ff_lp_bjt,NaN,0.5452


In [3]:
def listOneHotEncoder(items) :
    
    # Finding  unique items
    unique_items = list(set(items))

    # Length of the given list
    no_items = len(items)

    # We will be returning a numpy array
    encoded = np.zeros((no_items, 1))

    for i, item in enumerate(items) :
        encoded[i] = unique_items.index(item)

    return encoded

In [4]:
class CadenceDataset(Dataset) :

    def __init__(self) :

        # Loading data
        data = pd.read_csv('./team_26.csv')

        # The following 5 values are the inputs
        vsup = np.array(data['vsup']).reshape(-1, 1)
        temperature = np.array(data['temperature']).reshape(-1, 1)
        model1 = listOneHotEncoder(data['l55lp_v0101.lib.scs'])
        model2 = listOneHotEncoder(data['l55lp_v0101.lib.scs.1'])
        model3 = listOneHotEncoder(data['l55lp_v0101.lib.scs.2'])

        # Concatenating them
        self.x = np.concatenate((vsup, temperature, model1, model2, model3), axis=1)
        
        # Corresponding outputs
        self.y = np.array(data['VDC("/net13")']).reshape(-1, 1)

        # Scaling the data
        self.x = StandardScaler().fit_transform(self.x)

        # Converting both to tensor
        self.x = torch.from_numpy(self.x)
        self.y = torch.from_numpy(self.y)

        # Saving the number of samples
        self.length = len(self.x)

    def __getitem__(self, index) :
        return self.x[index].float(), self.y[index].float()

    def __len__(self) :
        return self.length

In [5]:
dataset = CadenceDataset()

In [6]:
# Splitting the data for training and testing
train_dataset, test_dataset = random_split(dataset, [2500, 1172])

In [7]:
# Generating dataloaders
train_dataloader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=True)

### Linear Regression

In [8]:
X_train = train_dataset[:][0]
Y_train = train_dataset[:][1]

X_test = test_dataset[:][0]
Y_test = test_dataset[:][1]

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [10]:
LinearModel = LinearRegression()

reg = LinearModel.fit(X_train, Y_train)

In [11]:
def MSE(a, b) :
    return np.mean((a-b)**2)

In [12]:
# Calculating the accuracy using R2 metric
pred = reg.predict(X_test)

print(f"R2 score for the linear regression model is {r2_score(Y_test, pred)}")

R2 score for the linear regression model is 0.8692684968436928


### Multi Layer Perceptron

In [13]:
# Defining the model

model = nn.Sequential(
    nn.Linear(5, 20),
    nn.ReLU(),
    nn.Linear(20, 20),
    nn.ReLU(),
    nn.Linear(20, 20),
    nn.ReLU(),
    nn.Linear(20, 1)
)

In [14]:
# Defining optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Defining scheduler
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

# Defining criterion
criterion = nn.MSELoss()

In [15]:
# Training function
def train(optimizer, scheduler, criterion, epochs=10) :
    for e in range(epochs):
        train_loss = 0.0
        
        for data, labels in tqdm(train_dataloader):
            # Transfer Data to GPU if available
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()
            
            # Clear the gradients
            optimizer.zero_grad()
            # Forward Pass
            target = model(data)
            # Find the Loss
            loss = criterion(target,labels)
            # Calculate gradients 
            loss.backward()
            # Update Weights
            optimizer.step()
            # Calculate Loss
            train_loss += loss.item()
        
        scheduler.step()

        # print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_dataloader)}')

In [16]:
train(optimizer, scheduler, criterion, 100)

100%|██████████| 625/625 [00:00<00:00, 1135.52it/s]


In [17]:
# Finding out the predictions
pred = model(X_test)

# Calculating the R2 score
r2_score(Y_test.numpy(), pred.detach().numpy())

0.9963606160465203